In [1]:
% load_ext autoreload
% autoreload 2

import numpy as np
import scipy as sp
from drift_qec.lifetime import *
from os import getpid
from time import time
np.set_printoptions(precision=5, linewidth=120)

In [3]:
fname = str(getpid()) + "-" + str(time()) + ".csv"
channel = Channel(p=0.001, n=15,
                  kx=0.99998, ky=0.00001, kz=0.00001,
                  Q=np.linalg.qr(np.random.random((3, 3)))[0],
                  stacklength=2001,
                  Vdecayfn = lambda V, t: V / np.sqrt(float(t)))

with open(fname, "w") as f:
    f.write("# Channel\n")
    f.write("# p: {}\n".format(channel.p))
    f.write("# kx: {}\n".format(channel.kx))
    f.write("# ky: {}\n".format(channel.ky))
    f.write("# kz: {}\n".format(channel.kz))
    f.write("# stacklength: {}\n".format(channel.stacklength))
    f.write("# q1: {}\n".format(channel.Q[:, 0]))
    f.write("# q2: {}\n".format(channel.Q[:, 1]))
    f.write("# q3: {}\n".format(channel.Q[:, 2]))
    f.write("fname,error_rate,t," \
            + "err1,err2,err3," \
            + "d1,d2,d3," \
            + "pxhat,pyhat,pzhat,"
            + "|V|,"
            + "C_M_Fro,q_qhat_2\n")

failflag = False
while not failflag:
    failflag, err1, err2, err3 = channel.step()
    with open(fname, "w") as f:
        # fname, error_rate, t
        f.write("{},{},{}".format(fname, channel.p, channel.time))
        
        # d1, d2, d3
        f.write("{},{},{}".format(channel.d1, channel.d2, channel.d3))
        
        # pxhat, pyhat, pzhat
        pxhat, pyhat, pzhat = np.real(np.linalg.eig(channel.M)[0])
        f.write("{},{},{}".format(pxhat, pyhat, pzhat))
        
        # |V|
        f.write("{},".format(np.sum(channel.V)))
        
        # C_M_Fro
        f.write("{},".format(np.linalg.norm(channel.C - channel.M)))
        
        # q_qhat_2
        Qhat = np.linalg.svd(channel.M)[0]
        f.write("{}".format(np.linalg.norm(channel.Q[:, 0] - Qhat[:, 0])))
channel.time

14170

In [4]:
fname

'25759-1460942966.31.csv'